# Desafio 3 - Maratona Behind the Code 2021

## Bibliotecas

In [1]:
from pathlib import Path

import pandas as pd

## Configurações

In [2]:
seed = 42
data_path = Path("../desafio/assets/data/")

## Dados

### Obtenção dos dados

In [3]:
retail = pd.read_csv(data_path / "RetailBankEFG.csv", index_col="ID")
retail_cols = list(retail.columns)
print(retail_cols)

['Financiamento Casa', 'Financiamento Carro', 'Emprestimo _pessoal', 'Emprestimo _consignado', 'Emprestimo _limite_especial', 'Emprestimo _educacao', 'Emprestimo _viagem', 'Investimento CDB', 'Investimento Fundos']


In [4]:
investment = pd.read_csv(data_path / "InvestmentBankCDE.csv", index_col="ID")
investment_cols = list(investment.columns)
print(investment_cols)

['Investimento Fundos_cambiais', 'Investimento Fundos_commodities', 'Investimento LCI', 'Investimento LCA', 'Investimento Poupanca', 'Investimento Fundos Multimercado', 'Investimento Tesouro Direto']


In [5]:
insurance = pd.read_csv(data_path / "InsuranceCompanyABC.csv", index_col="ID")
insurance_cols = list(insurance.columns)
print(insurance_cols)

['Idade', 'Renda', 'Regiao', 'Genero', 'seguro auto', 'seguro vida Emp', 'seguro vida PF', 'Seguro Residencial']


In [6]:
answers = pd.read_csv(data_path / "ANSWERS.csv")
print(list(answers.columns))

['ID', 'Idade', 'Renda', 'Regiao', 'Genero', 'seguro auto', 'seguro vida Emp', 'seguro vida PF', 'Seguro Residencial', 'Investimento Fundos_cambiais', 'Investimento Fundos_commodities', 'Investimento LCI', 'Investimento LCA', 'Investimento Poupanca', 'Investimento Fundos Multimercado', 'Investimento Tesouro Direto', 'Financiamento Casa', 'Financiamento Carro', 'Emprestimo _pessoal', 'Emprestimo _consignado', 'Emprestimo _limite_especial', 'Emprestimo _educacao', 'Emprestimo _viagem', 'Investimento CDB', 'Investimento Fundos', 'RECOMMENDATION_1', 'CONFIDENCE_1', 'RECOMMENDATION_2', 'CONFIDENCE_2', 'RECOMMENDATION_3', 'CONFIDENCE_3']


In [7]:
assert insurance.index.equals(investment.index) and insurance.index.equals(retail.index)

df = pd.concat([insurance, investment, retail], axis=1).reset_index()
target_cols = ['RECOMMENDATION_1', 'CONFIDENCE_1', 'RECOMMENDATION_2', 'CONFIDENCE_2', 'RECOMMENDATION_3', 'CONFIDENCE_3']

### Dados de exemplo

In [8]:
with pd.option_context("display.max_columns", None):
    display(df.sample(2, random_state=seed))

,ID,Idade,Renda,Regiao,Genero,seguro auto,seguro vida Emp,seguro vida PF,Seguro Residencial,Investimento Fundos_cambiais,Investimento Fundos_commodities,Investimento LCI,Investimento LCA,Investimento Poupanca,Investimento Fundos Multimercado,Investimento Tesouro Direto,Financiamento Casa,Financiamento Carro,Emprestimo _pessoal,Emprestimo _consignado,Emprestimo _limite_especial,Emprestimo _educacao,Emprestimo _viagem,Investimento CDB,Investimento Fundos
5032,79717.0,29.944,7511.742,CO,1.0,F,F,F,F,T,T,T,T,T,T,T,F,F,F,F,F,F,F,F,F
33,20858.0,36.170,5276.354,SE,0.0,T,F,F,F,T,T,F,T,F,F,F,T,T,F,F,F,T,F,F,F


### Tipos de dados

In [9]:
df['ID'] = df['ID'].astype(int)
df['Genero'] = df['Genero'].astype(bool)  # Whether the client is of a given gender (which one?)
df['Regiao'] = df['Regiao'].astype('category')

for col in df.columns:
    if df[col].iloc[0] in ['F', 'T'] and df[col].nunique() == 2:
        df[col] = df[col].map({'T': True, 'F': False})

### Informações gerais

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10082 entries, 0 to 10081
Data columns (total 25 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   ID                                10082 non-null  int64   
 1   Idade                             10082 non-null  float64 
 2   Renda                             10082 non-null  float64 
 3   Regiao                            10082 non-null  category
 4   Genero                            10082 non-null  bool    
 5   seguro auto                       10082 non-null  bool    
 6   seguro vida Emp                   10082 non-null  bool    
 7   seguro vida PF                    10082 non-null  bool    
 8   Seguro Residencial                10082 non-null  bool    
 9   Investimento Fundos_cambiais      10082 non-null  bool    
 10  Investimento Fundos_commodities   10082 non-null  bool    
 11  Investimento LCI                  10082 non-null  bool

Neste conjunto de dados:
- Há tanto variáveis numéricas quanto categóricas/booleanas
- Não há dados faltantes

### Variáveis categóricas e booleanas

In [12]:
non_numeric_cols = list(df.select_dtypes(include=['category', bool]).columns)
with pd.option_context("display.max_columns", None):
    display(df.describe(include=['category', bool]))

,Regiao,Genero,seguro auto,seguro vida Emp,seguro vida PF,Seguro Residencial,Investimento Fundos_cambiais,Investimento Fundos_commodities,Investimento LCI,Investimento LCA,Investimento Poupanca,Investimento Fundos Multimercado,Investimento Tesouro Direto,Financiamento Casa,Financiamento Carro,Emprestimo _pessoal,Emprestimo _consignado,Emprestimo _limite_especial,Emprestimo _educacao,Emprestimo _viagem,Investimento CDB,Investimento Fundos
count,10082,10082,10082,10082,10082,10082,10082,10082,10082,10082,10082,10082,10082,10082,10082,10082,10082,10082,10082,10082,10082,10082
unique,5,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,SE,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
freq,4557,5531,7268,6602,7566,6729,6602,6603,6658,7068,5834,7530,7069,8947,7580,8023,8497,8470,8494,7594,5068,6699


In [13]:
df['ID'].astype('category').describe().to_frame()

,ID
count,10082
unique,9530
top,14391
freq,4


Observações:
- Somente para duas das variáveis booleanas o valor `True` é o mais frequente: `Genero` e `Investimento CDB`
- Alguns índices (`ID`) ocorrem mais de uma vez

### Variáveis numéricas

In [14]:
numeric_cols = list(df.select_dtypes(include=float).columns)
with pd.option_context("display.max_columns", None):
    display(df.describe(include=float))

,Idade,Renda
count,10082.000000,10082.000000
mean,38.461186,7783.396381
std,12.212422,7303.024782
min,18.009000,2312.000000
25%,28.925500,6971.313000
50%,37.229500,7522.556000
75%,46.639500,8061.152000
max,85.605000,580000.000000
